# Berlin tech job market - database connection and pre-processing

Objective: Access data stored in the postgres SQL database and pre-process text
- Access SQL database and save to file
- Manually insert job_class data into xls file
- This will be the basis for further text pre-processing


In [34]:
# import statements
import pandas as pd
import numpy as np
import sqlalchemy as db

from googletrans import Translator



In [2]:
# connect to Postgres
DATABASE_USER = 'postgres'
DATABASE_PASSWORD = 'titanic99'
DATABASE_HOST = 'localhost'
DATABASE_PORT = '5432'
DATABASE_DB_NAME = 'linkedin'

In [13]:
engine = db.create_engine(f'postgres://{DATABASE_USER}:{DATABASE_PASSWORD}\
@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_DB_NAME}')
connection = engine.connect()
metadata = db.MetaData()

In [14]:
# connect to table specifications
specifications = db.Table('Specifications', metadata, autoload=True, autoload_with=engine)

In [15]:
# Print the column names
columns = specifications.columns.keys()
print(columns)

['url', 'language', 'job_title', 'job_location', 'company_name', 'Job_Id', 'ID', 'job_ad_duration', 'job_applications', 'seniority_level', 'industry', 'employment_type', 'job_functions', 'job_text', 'date_added', 'date_scraped']


In [16]:
columns_dict={'url', 'language', 'job_title', 'job_location', 'company_name', 'Job_Id', 'ID', 'job_ad_duration', 'job_applications', 'seniority_level', 'industry', 'employment_type', 'job_functions', 'job_text', 'date_added', 'date_scraped'}


In [17]:
# Print full table metadata
print(repr(metadata.tables['Specifications']))

Table('Specifications', MetaData(bind=None), Column('url', VARCHAR(), table=<Specifications>), Column('language', VARCHAR(), table=<Specifications>), Column('job_title', VARCHAR(), table=<Specifications>), Column('job_location', VARCHAR(), table=<Specifications>), Column('company_name', VARCHAR(), table=<Specifications>), Column('Job_Id', BIGINT(), table=<Specifications>, nullable=False), Column('ID', BIGINT(), table=<Specifications>, primary_key=True, nullable=False), Column('job_ad_duration', VARCHAR(), table=<Specifications>), Column('job_applications', VARCHAR(), table=<Specifications>), Column('seniority_level', VARCHAR(), table=<Specifications>), Column('industry', VARCHAR(), table=<Specifications>), Column('employment_type', VARCHAR(), table=<Specifications>), Column('job_functions', TEXT(), table=<Specifications>), Column('job_text', TEXT(), table=<Specifications>), Column('date_added', TIMESTAMP(), table=<Specifications>), Column('date_scraped', TIMESTAMP(), table=<Specificati

In [18]:
# define query to collect all data from Specifications table
query = db.select([specifications])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [19]:
df = pd.DataFrame(ResultSet)

In [20]:
# rename columns
df.rename(columns={0:'url', 1:'language', 2:'job_title', 3:'job_location', 4:'company_name', 5:'Job_Id', 6:'ID', 7:'job_ad_duration', 8:'job_applications', 9:'seniority_level', 10:'industry', 11:'employment_type', 12:'job_functions', 13:'job_text', 14:'date_added', 15:'date_scraped'},
             inplace=True)


In [25]:
# download dataframe as xls in order to manually include job_class as y labels (based on job_title)
df.to_excel('99_data/200810_manual_job_class.xlsx')
# --> manually label the data for job_class (data scientist, data engineer, data analyst)

In [27]:
# load xls data from file
df = pd.read_excel('99_data/200810_manual_job_class_final.xlsx', sheet_name='Sheet1')
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [31]:
df

,url,language,job_title,job_class,job_location,company_name,Job_Id,ID,job_ad_duration,job_applications,seniority_level,industry,employment_type,job_functions,job_text,date_added,date_scraped
0,https://de.linkedin.com/jobs/view/1903068784,NaN,Data Analyst Online Marketing (m/w/d),Data Analyst,Berlin,advalyze GmbH,1903068784,2951,Vor 6 Stunden,45,Berufseinstieg,Management-Beratung,Vollzeit,IT,Die advalyze GmbH sucht ab sofort einen Data A...,2020-07-27 12:28:26.182,2020-07-27 18:28:23.797
1,https://de.linkedin.com/jobs/view/1967529202,NaN,(Online-)Weiterbildung für Naturwissenschaftle...,Data Scientist,Berlin,CQ Beratung+Bildung GmbH,1967529202,2952,Vor 6 Stunden,25,Berufseinstieg,"Chemie, Biotechnologie, Pharmazie",Vollzeit,Personalwesen,Erlernen Sie schnell und effektiv große naturw...,2020-07-27 12:28:28.071,2020-07-27 18:28:28.086
2,https://de.linkedin.com/jobs/view/1966811872,NaN,Absolventen der Physik oder Mathematik als Tra...,Data Scientist,Berlin,Heringer Consulting GmbH,1966811872,2953,Vor 14 Stunden,25,Keine Angabe,"IT und Services, Computer-Software, Internet",Vollzeit,Sonstiges,ABSOLVENTEN DER PHYSIK ODER MATHEMATIK ALS TRA...,2020-07-27 04:28:29.998,2020-07-27 18:28:30.001
3,https://de.linkedin.com/jobs/view/1966628233,NaN,"Data Scientist (m/w/d) - Ingenieur, Entwicklun...",Data Scientist,Berlin,First Sensor,1966628233,2954,Vor 17 Stunden,25,Berufseinstieg,"Elektro-/Elektronik-Herstellung, Maschinenbau,...",Vollzeit,"Ingenieurwesen, IT","Ihre Aufgaben Analyse, Darstellung und statist...",2020-07-27 01:28:32.013,2020-07-27 18:28:32.036
4,https://de.linkedin.com/jobs/view/1966592863,NaN,CC04-BIH-27.20 Data Engineer / Data Scientist ...,Data Scientist,Berlin,Charité,1966592863,2955,Vor 18 Stunden,25,Berufseinstieg,"Forschung, Personalberatung & -vermittlung, Kr...",Vollzeit,IT,UnternehmensbeschreibungDie Charité – Universi...,2020-07-27 00:28:33.862,2020-07-27 18:28:33.886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,https://de.linkedin.com/jobs/view/1959095674,NaN,Data Engineer (f/m/d),Data Engineer,Berlin,Kolibri Games,1959095674,4061,Vor 3 Tagen,26,Berufserfahren,"Unterhaltungselektronik, Unterhaltung, Compute...",Vollzeit,"IT, Ingenieurwesen",Three years ago in a student flat in Karlsruhe...,2020-08-07 13:06:38.667,2020-08-10 13:06:38.689
965,https://de.linkedin.com/jobs/view/1982989873,NaN,Data Infrastructure Engineer (m/f/d),Data Engineer,Berlin,PubNative,1982989873,4063,Vor 3 Tagen,25,Berufseinstieg,"IT und Services, Computer-Software, Internet",Vollzeit,IT,Verve™ uses Movement Science™ to successfully ...,2020-08-07 13:06:42.285,2020-08-10 13:06:42.289
966,https://de.linkedin.com/jobs/view/1982438292,NaN,BI Data Engineer (m/w/d) in Berlin gesucht,Data Engineer,Berlin,Apriori,1982438292,4073,Vor 3 Tagen,25,Berufseinstieg,"IT und Services, Computer-Software, Pharmazie",Vollzeit,"Geschäftsentwicklung, Vertrieb","Gemeinsam mit meinem Kunden, einem erfolgreich...",2020-08-07 13:07:00.994,2020-08-10 13:07:00.996
967,https://de.linkedin.com/jobs/view/1982436567,NaN,Database Engineer (m/w/x) für beständiges e-He...,Data Engineer,Berlin,Apriori,1982436567,4074,Vor 3 Tagen,25,Berufseinstieg,"IT und Services, Computer-Software, Pharmazie",Vollzeit,IT,Unser Klient ist ein weltweit führendes Untern...,2020-08-07 13:07:03.182,2020-08-10 13:07:03.189


## Translation with googletrans - Iterate over job text 

In [35]:
translator = Translator()

In [36]:
# iterate over dataframe
def translate():
    lang=[]
    title=[]
    text_new=[]
    text2_new=[]
    
    # detect language
    for value in df['job_text']:
        result=translator.detect(value)
        lang.append(result.lang)
    df['lang_detect'] = lang
    
    
    # iterate over dataframe to translate job_title
    for value in df['job_title']:
        text_title=translator.translate(value,src='de', dest='en')
        title.append(text_title.text)
    df['job_title_translated']=title
    
    # iterate over dataframe to translate job_text
    for value in df['job_text']:
        text_trans=translator.translate(value, src='de', dest='en')
        text_new.append(text_trans.text)
    df['job_text_translated']=text_new
    
    # iterate over dataframe to translate job_text_translate 2nd time
    for value in df['job_text_translated']:
        text_trans2=translator.translate(value, src='de', dest='en')
        text2_new.append(text_trans2.text)
    df['job_text_translated2']=text2_new

In [ ]:
translate()

In [ ]:
# look at data frame
df

## Text preprocessing

In [ ]:
# functions neccessary for stemming and lemmatizing text
# Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.

def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
# function for text preprocessing
def preprocess_text():

    # concatenate job_title_translated with job_text_translated
    df['text_concat']= df['job_title_translated'] + df['job_text_translated2']
    #lowercasing the job_text
    df["text_lower"] = df.text_concat.str.lower()
    #remove \n
    df["text_n"] = df.text_lower.str.replace("\\n", " ")
    #remove punctuation and tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    df["text_tokens"] = df.apply(lambda row: tokenizer.tokenize(row['text_n']), axis=1)
    #remove stopwords
    stop_words = stopwords.words('english')
    df['text_stop'] = df['text_tokens'].apply(lambda x: [item for item in x if item not in stop_words])
    #merge tokens back into string text
    df['text_merge']=[" ".join(txt) for txt in df["text_stop"].values]
    #create bi-grams
    df["text_bigrams"] = df["text_stop"].apply(lambda row: list(ngrams(row, 2)))
    
    lemma_text=[]
    for value in df['text_merge']:
        preprocess_text=preprocess(value)
        lemma_text.append(preprocess_text)
    df['lemma_text']=lemma_text

In [ ]:
preprocess_text()

In [ ]:
# look at preprocesses dataframe
df